In [1]:
# Import required libraries
import numpy as np
import pandas as pd

# Load the dataset files
train_data = pd.read_csv('/content/drive/MyDrive/IDL/Assignment#2/atis_intents_train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/IDL/Assignment#2/atis_intents_test.csv')
full_data = pd.read_csv('/content/drive/MyDrive/IDL/Assignment#2/atis_intents.csv')

# Display the first few rows of each dataset to check the structure
print("Training Data:")
print(train_data.head())

print("\nTesting Data:")
print(test_data.head())


Training Data:
        atis_flight  \
0       atis_flight   
1  atis_flight_time   
2      atis_airfare   
3      atis_airfare   
4       atis_flight   

   i want to fly from boston at 838 am and arrive in denver at 1110 in the morning  
0   what flights are available from pittsburgh to...                                
1   what is the arrival time in san francisco for...                                
2            cheapest airfare from tacoma to orlando                                
3   round trip fares from pittsburgh to philadelp...                                
4   i need a flight tomorrow from columbus to min...                                

Testing Data:
    atis_flight  \
0  atis_airfare   
1   atis_flight   
2   atis_flight   
3   atis_flight   
4   atis_flight   

   i would like to find a flight from charlotte to las vegas that makes a stop in st. louis  
0   on april first i need a ticket from tacoma to...                                         
1   on april first

In [3]:
# Display column names in the training and testing datasets
print("Training Data Columns:", train_data.columns)
print("Testing Data Columns:", test_data.columns)


Training Data Columns: Index(['atis_flight', ' i want to fly from boston at 838 am and arrive in denver at 1110 in the morning'], dtype='object')
Testing Data Columns: Index(['atis_flight', ' i would like to find a flight from charlotte to las vegas that makes a stop in st. louis'], dtype='object')


In [8]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Extract columns explicitly
X_train = train_data.iloc[:, 1]  # Queries (2nd column, index 1)
y_train = train_data.iloc[:, 0]  # Intents (1st column, index 0)

X_test = test_data.iloc[:, 1]  # Queries (2nd column, index 1)
y_test = test_data.iloc[:, 0]  # Intents (1st column, index 0)

# Label encoding for intents
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Tokenize and convert text to sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)  # Fit tokenizer on training data only

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences for uniform input size
X_train_pad = pad_sequences(X_train_seq, padding='post')
X_test_pad = pad_sequences(X_test_seq, padding='post')

# Display shapes
print(f"Training data shape: {X_train_pad.shape}")
print(f"Test data shape: {X_test_pad.shape}")
print(f"Number of unique intents: {len(label_encoder.classes_)}")


Training data shape: (4833, 46)
Test data shape: (799, 30)
Number of unique intents: 8


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

# Define model parameters
vocab_size = len(tokenizer.word_index) + 1  # Total vocabulary size (+1 for padding token)
max_len = X_train_pad.shape[1]  # Sequence length from padded data
num_classes = len(label_encoder.classes_)  # Number of unique intents

# Verify parameters
print(f"Vocab Size: {vocab_size}, Max Sequence Length: {max_len}, Number of Classes: {num_classes}")

# Build the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(32)),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.build(input_shape=(None, max_len))  # Explicitly define the input shape
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Display the model summary
model.summary()


Vocab Size: 872, Max Sequence Length: 46, Number of Classes: 8


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 46, 128)             │         111,616 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (None, 46, 128)             │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 46, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ (None, 64)                  │          41,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 8)                   │             264 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 253,992 (992.16 KB)

 Trainable params: 253,992 (992.16 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Train the model
history = model.fit(
    X_train_pad,
    y_train,
    validation_split=0.2,  # 20% of training data for validation
    epochs=10,
    batch_size=32,
    verbose=1
)

# Save the model for future use
model.save('intent_classification_model.keras')


Epoch 1/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 15s 123ms/step - accuracy: 0.9625 - loss: 0.1218 - val_accuracy: 0.9669 - val_loss: 0.1439
Epoch 2/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 15s 124ms/step - accuracy: 0.9651 - loss: 0.0987 - val_accuracy: 0.9659 - val_loss: 0.1602
Epoch 3/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 20s 120ms/step - accuracy: 0.9729 - loss: 0.0956 - val_accuracy: 0.9710 - val_loss: 0.1495
Epoch 4/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 21s 124ms/step - accuracy: 0.9722 - loss: 0.0966 - val_accuracy: 0.9741 - val_loss: 0.1586
Epoch 5/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 15s 122ms/step - accuracy: 0.9758 - loss: 0.0707 - val_accuracy: 0.9752 - val_loss: 0.1711
Epoch 6/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 21s 123ms/step - accuracy: 0.9724 - loss: 0.0982 - val_accuracy: 0.9741 - val_loss: 0.1648
Epoch 7/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 15s 122ms/step - accuracy: 0.9839 - loss: 0.0538 - val_accuracy: 0.9772 - val_loss: 0.2001
Epoch 8/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 15s 124ms/step - accuracy: 0.9809 - loss: 0

In [19]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on test data
y_pred = model.predict(X_test_pad)
y_pred_classes = y_pred.argmax(axis=1)  # Get predicted class indices

# Calculate test accuracy
accuracy = accuracy_score(y_test, y_pred_classes)
print(f"Test Accuracy: {accuracy:.2f}")

# Generate classification report
report = classification_report(
    y_test,
    y_pred_classes,
    target_names=label_encoder.classes_
)
print(report)


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Test Accuracy: 0.97
                     precision    recall  f1-score   support

  atis_abbreviation       0.94      1.00      0.97        33
      atis_aircraft       0.80      0.89      0.84         9
       atis_airfare       0.90      0.90      0.90        48
       atis_airline       1.00      0.97      0.99        38
        atis_flight       0.99      0.98      0.99       631
   atis_flight_time       0.50      1.00      0.67         1
atis_ground_service       1.00      1.00      1.00        36
      atis_quantity       0.38      1.00      0.55         3

           accuracy                           0.97       799
          macro avg       0.81      0.97      0.86       799
       weighted avg       0.98      0.97      0.98       799



In [20]:
def predict_intent(query):
    # Preprocess the query
    seq = tokenizer.texts_to_sequences([query])
    pad_seq = pad_sequences(seq, maxlen=max_len, padding='post')

    # Predict intent
    prediction = model.predict(pad_seq)
    predicted_class = prediction.argmax(axis=1)[0]
    intent = label_encoder.inverse_transform([predicted_class])[0]

    return intent

# Example usage
user_query = "show me flights from New York to Los Angeles"
predicted_intent = predict_intent(user_query)
print(f"Predicted Intent: {predicted_intent}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Predicted Intent: atis_flight
